In [3]:
import redis
from redis.sentinel import Sentinel
import urllib2
import urllib
import Queue
import hashlib

In [4]:
def startSentinelListner():
    p = REDIS_SENTINEL.pubsub()
    c hannels = ['+sdown','-sdown']
    p.psubscribe('*')

    for message in p.listen():
        handleMessage(message)
        
dtSentinelInit()
startSentinelListner()

['masterP_30001', 'masterP_30003', 'masterP_30002', 'masterP_30005', 'masterP_30004', 'masterP_30006']
HASH_TO_BACK_MAP updated: ['127.0.0.1:30001', '127.0.0.1:30002', '127.0.0.1:30003', '127.0.0.1:30004', '127.0.0.1:30005', '127.0.0.1:30006']
handleLeave():
['127.0.0.1:30003', '127.0.0.1:30002', '127.0.0.1:30005', '127.0.0.1:30004', '127.0.0.1:30006']
HASH_TO_BACK_MAP updated: ['127.0.0.1:30002', '127.0.0.1:30003', '127.0.0.1:30004', '127.0.0.1:30005', '127.0.0.1:30006']
{'pattern': '*', 'type': 'pmessage', 'channel': '-sdown', 'data': 'master masterP_30001 127.0.0.1 30001'}
handleJoin(): master masterP_30001 127.0.0.1 30001
['127.0.0.1:30001', '127.0.0.1:30003', '127.0.0.1:30002', '127.0.0.1:30005', '127.0.0.1:30004', '127.0.0.1:30006']
HASH_TO_BACK_MAP updated: ['127.0.0.1:30001', '127.0.0.1:30002', '127.0.0.1:30003', '127.0.0.1:30004', '127.0.0.1:30005', '127.0.0.1:30006']


KeyboardInterrupt: 

In [1]:
REDIS_SENTINEL = None
MSG_QUEUE = None
NUMBACKUPS = 2
HASH_TO_BACK_MAP = None

def dtSentinelInit():
    global REDIS_SENTINEL
    global MSG_QUEUE
    global HASH_TO_BACK_MAP
    REDIS_SENTINEL = redis.StrictRedis(host='localhost', port=40001)
    print REDIS_SENTINEL.sentinel_masters().keys()
    HASH_TO_BACK_MAP = {}
    updateHashToBackMap()
    REDIS_SENTINEL.sentinel_masters()['masterP_30001']['ip']
    MSG_QUEUE = Queue.Queue(maxsize=50)

    
def getKeysInMaster(back):
    addr = back.split(":")[0]
    port = back.split(":")[1]
    r = redis.StrictRedis(host=addr, port=port, db=0)
    r.keys()
    
def getKeysInRange(addr1,addr2):
    # copy keys in range from addr1 till addr2 - 1. Keys of addr2 are not included.
    hkeys = [sorted(HASH_TO_BACK_MAP.keys())]
    k1, k2 = hash(addr1), hash(addr2)
    p1, p2 = hkeys.index(hash(addr1)),hkeys.index(hash(addr2))
    backs = []
    while p1 != p2:
        backs.append(HASH_TO_BACK_MAP[hkeys[p1]])
        p1 = (p1+1)%len(keys)
    
    print "Getting keys from backs:", backs
    allkeys = []
    for b in backs:
        allkeys.append(getKeysInMaster(b))
    
    return None

def getMastersSorted():
    keys = sorted(HASH_TO_BACK_MAP.keys())
    
def hash(key):
    return int(hashlib.sha1(key).hexdigest(), 16)

def updateHashToBackMap():
    ml = getMastersList()
    HASH_TO_BACK_MAP.clear()
    for m in ml:
        HASH_TO_BACK_MAP[hash(m)] = m 
    print "HASH_TO_BACK_MAP updated:", sorted(HASH_TO_BACK_MAP.values())

def getNextNBackends(server_addr, N=NUMBACKUPS):
    key = hash(server_addr)
    keys = sorted(HASH_TO_BACK_MAP.keys())
    index = keys.index(key)
    out = []
#     for k in keys:
#         print k, HASH_TO_BACK_MAP[k]
#     print "hash:", key, ":",server_addr
#     print "index:", index
    for i in range(1,N+1):
        # we are selecting N backends starting with the backend following the server_addr
        # the output list does not contain server_addr
        if keys[(index + i)%len(keys)] in HASH_TO_BACK_MAP and HASH_TO_BACK_MAP[keys[(index + i)%len(keys)]] not in out:
            out.append(HASH_TO_BACK_MAP[keys[(index + i)%len(keys)]])
    print "getNextNBackends for addr : ",server_addr," returning : {",out,"}"
    return out

def getPrevNBackends(server_addr, N=NUMBACKUPS):
    key = hash(server_addr)
    keys = sorted(HASH_TO_BACK_MAP.keys())
    index = keys.index(key)
    out = []
    for i in range(1,N+1):
        # we are selecting N backends starting with the backend preceeding the server_addr
        # the output list does not contain server_addr
        if keys[(index - i)%len(keys)] in HASH_TO_BACK_MAP and HASH_TO_BACK_MAP[keys[(index - i)%len(keys)]] not in out:
            out.append(HASH_TO_BACK_MAP[keys[(index - i)%len(keys)]])
    print "getNextNBackends for addr : ",server_addr," returning : {",out,"}"
    return out

def getMastersList():
    return [REDIS_SENTINEL.sentinel_masters()[m]['ip'] + ':' + str(REDIS_SENTINEL.sentinel_masters()[m]['port']) for m in REDIS_SENTINEL.sentinel_masters() if not REDIS_SENTINEL.sentinel_masters()[m]['is_sdown']]

def sendMasterList(ml):
    for url in ml:
        ip,port = url.split(':')
        if port == '30001':
            sendListAsPost('http://'+ip+':'+str(int(port) + 20000), ml)
            break

def handleJoin(msg):
    print msg
    print 'handleJoin():', msg['data']
    ml = getMastersList()
    print ml
    updateHashToBackMap()
#     getNextNBackends(ml[0])
#     getPrevNBackends(ml[0])
#     sendMasterList(ml)
    
    
def handleLeave(msg):
    print 'handleLeave():'#, msg['data']
    ml = getMastersList()
    print ml
    updateHashToBackMap()
#     getNextNBackends(ml[0])
#     getPrevNBackends(ml[0])
#     sendMasterList(ml)
    
def handleMessage(message):
    
    t = message['channel']
#     print message['data'], t
    if t == '+sdown':
        handleLeave(message)
    elif t == '-sdown':
        handleJoin(message)
    else:
#         print "Unknown channel message:", message
        t = None
def sendListAsPost(url, slist):
    url = url + '/update_hash_to_back_map?'
    print url, slist
    s = ''
    for i in slist: s = s + ',' + i
    s = s[1:]
    data = urllib.urlencode({'addrs' : s})
#     print 'encoded data', data
    response = urllib2.urlopen(url+data)
    print "POST Request:" + url+data
    result = response.read()
    return result